In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt

import numpy as np
import pandas as pd; pd.options.mode.chained_assignment = None

import APIs.Bloomberg as ba
import Weather.W_USA as wu

import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import plotly.express as px

In [ ]:
runs_df=ba.print_finished_weather_runs_lengths()

In [9]:
runs_df=ba.latest_weather_run_df()

Invalid parameter passed: CID: {[ valueType=AUTOGEN classId=0 value=85 ]}
RequestId: 1d1d47f8-7e06-488d-8d5b-7f9fc45c2ed8
GridResponse = {
    NumOfFields = 0
    NumOfRecords = 0
    ColumnTitles[] = {
    }
    DataRecords[] = {
        DataRecords = {
            DataFields[] = {
            }
        }
    }
    ReachMax = true
    Error = "No weather data points found."
    SequenceNumber = 0
}

Invalid parameter passed: CID: {[ valueType=AUTOGEN classId=0 value=86 ]}
RequestId: 638c5085-d9d2-4ec0-a907-638902a07a47
GridResponse = {
    NumOfFields = 0
    NumOfRecords = 0
    ColumnTitles[] = {
    }
    DataRecords[] = {
        DataRecords = {
            DataFields[] = {
            }
        }
    }
    ReachMax = true
    Error = "No weather data points found."
    SequenceNumber = 0
}

Invalid parameter passed: CID: {[ valueType=AUTOGEN classId=0 value=87 ]}
RequestId: f5b58aa2-d344-4406-bb87-672b16ed5954
GridResponse = {
    NumOfFields = 0
    NumOfRecords = 0
    ColumnTi

In [10]:
runs_df

,model,model_type,Run,Latest Available Run,Completed (%),Completed,of
model_full,,,,,,,
GFS Operational,GFS,Operational,2022-07-07 06:00:00,07 Jul 2022 06Z,100.0%,129,129
GFS Ensemble,GFS,Ensemble,2022-07-07 06:00:00,07 Jul 2022 06Z,89.2%,58,65
ECMWF Operational,ECMWF,Operational,2022-07-07 06:00:00,07 Jul 2022 06Z,100.0%,31,31
ECMWF Ensemble,ECMWF,Ensemble,2022-07-07 00:00:00,07 Jul 2022 00Z,100.0%,61,61


In [11]:
blp = ba.BLPInterface('//blp/exrsvc')

location = 'US_IL'
run=dt(2022,7,7)

run_str = run.strftime("%Y-%m-%dT%H:%M:%S")
overrides = {'location': location, 'fields':'TEMPERATURE|PRECIPITATION', 'model':'GFS','publication_date':run_str,'location_time':True,'type':'DETERMINISTIC'}

df_GFS = blp.bsrch('comdty:weather', overrides)
df_GFS

,Reported Time,Location Time,Temperature (°C),Precipitation (mm)
0,2022-07-07 00:00:00+00:00,2022-07-06T19:00:00-05:00,28.66,
1,2022-07-07 03:00:00+00:00,2022-07-06T22:00:00-05:00,24.27,0.91
2,2022-07-07 06:00:00+00:00,2022-07-07T01:00:00-05:00,22.54,0.4
3,2022-07-07 09:00:00+00:00,2022-07-07T04:00:00-05:00,21.90,2.02
4,2022-07-07 12:00:00+00:00,2022-07-07T07:00:00-05:00,22.85,0.99
...,...,...,...,...
124,2022-07-22 12:00:00+00:00,2022-07-22T07:00:00-05:00,20.30,0.13
125,2022-07-22 15:00:00+00:00,2022-07-22T10:00:00-05:00,24.57,0.01
126,2022-07-22 18:00:00+00:00,2022-07-22T13:00:00-05:00,27.54,0.0
127,2022-07-22 21:00:00+00:00,2022-07-22T16:00:00-05:00,28.78,0.05
